In [267]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sea
from matplotlib import pyplot as plt
from abrir_projeto import Projeto
pj = Projeto()
pj.abrir_projeto()
#### 1.A - Exibição da Documentação basilar para navegação.

def carregar():
        
        from pandas import DataFrame as pd

        global agnts_economicos
        agnts_economicos = pd(pj.df.agnts_economicos)
        global atividade_economicas_agnts
        atividade_economicas_agnts = pd(pj.df.atividade_economicas_agnts)
        global proj_renuncia_fiscal
        proj_renuncia_fiscal = pd(pj.df.proj_renuncia_fiscal)
        global obras_n_publi_fomento_direto
        obras_n_publi_fomento_direto = pd(pj.df.obras_n_publi_fomento_direto)
        global obras_n_publi_investimento_fsa
        obras_n_publi_investimento_fsa = pd(pj.df.obras_n_publi_investimento_fsa)
        global processos_prestacao_contas
        processos_prestacao_contas = pd(pj.df.processos_prestacao_contas)
        global investidores_proj_renuncia_fiscal
        investidores_proj_renuncia_fiscal = pd(pj.df.investidores_proj_renuncia_fiscal)
        global produtoras_independentes
        produtoras_independentes = pd(pj.df.produtoras_independentes)
        global produtores_obras_n_publi
        produtores_obras_n_publi = pd(pj.df.produtores_obras_n_publi)
        global proj_fsa_desembolso
        proj_fsa_desembolso = pd(pj.df.proj_fsa_desembolso)
        global relacao_grupos_economicos
        relacao_grupos_economicos = pd(pj.df.relacao_grupos_economicos)
        global salas_complexos
        salas_complexos = pd(pj.df.salas_complexos)    
carregar()


***Bem vindo(a) ao Projeto!***

Para ler documentação: "pj.doc.documentacao"



Criar Tabela Unificada - Agentes Econômicos

Vamos unificar essa tabela partindo do referencial - Agentes Econômicos

In [289]:
#Natureza Jurídica dos Agentes Econômicos

#Todo Agente Econômico tem CNPJ e Registro na Ancine
#Criando Dataframe merged Agentes Economicos Regulares x Atividades Economicas dos Agentes
agnts_atividades_economicas = pd.merge(agnts_economicos, atividade_economicas_agnts, on='REGISTRO_ANCINE', how='inner')
agnts_atividades_economicas = agnts_atividades_economicas[['REGISTRO_ANCINE', 'DATA_REGISTRO', 'RAZAO_SOCIAL_x', 'CNPJ_x',
       'DATA_CONSTITUICAO', 'UF', 'MUNICIPIO', 'NATUREZA_JURIDICA', 'ATIVIDADE_ECONOMICA','CLASSIFICACAO_ATIVIDADE', 'BRASILEIRO_INDEPENDENTE']]

agnts_atividades_economicas.rename(columns={'RAZAO_SOCIAL_x' : 'RAZAO_SOCIAL', 'CNPJ_x' : 'CNPJ'}, inplace=True) #corrigindo nomes das colunas

agnts_atividades_economicas = agnts_atividades_economicas[agnts_atividades_economicas['CLASSIFICACAO_ATIVIDADE'] == 1] # filtrando a colunaClassificação de atividades para deixar apenas as principais, evitando agentes duplicados

#Produtoras Independentes
produtoras_independentes = produtoras_independentes[['REGISTRO_ANCINE', 'CLASSIFICACAO_NIVEL_PRODUTORA']]
agnts_tabela_unificada = pd.merge(agnts_atividades_economicas, produtoras_independentes, how='outer', on='REGISTRO_ANCINE') # juntando tabela produtoras_independentes com agnts_atividades_aconomicas
# agnts_tabela_unificada['CLASSIFICACAO_NIVEL_PRODUTORA'].fillna('0.0', inplace=True) # Preenchendo valor nulo 
agnts_tabela_unificada['PRODUTORA_INDEPENDENTE'] = agnts_tabela_unificada['CLASSIFICACAO_NIVEL_PRODUTORA'].notna()

#Criando lista com as principais Atividades Econômicas da indústria cinematográfica

atividade_economica = ['Atividades de produção cinematográfica, de vídeos e de programas de televisão não especificadas anteriormente',
       'Atividades de pós-produção cinematográfica, de vídeos e de programas de televisão não especificadas anteriormente',
       'Produção de filmes para publicidade',
       'Atividades de produção de fotografias, exceto aérea e submarina',
       'Filmagem de festas e eventos',
       'Estúdios cinematográficos',
       'Atividades de gravação de som e de edição de música',
       'Produção musical',
       'Distribuição cinematográfica, de vídeo e de programas de televisão',
       'Serviços de mixagem sonora em produção audiovisual',
       'Atividades de sonorização e de iluminação',
       'Agências de publicidade',
       'Atividades de produção de fotografias aéreas e submarinas',
       'Atividades de exibição cinematográfica', 'Serviços de dublagem']


agnts_tabela_unificada = agnts_tabela_unificada.loc[agnts_tabela_unificada['ATIVIDADE_ECONOMICA'].isin(atividade_economica)]
agnts_tabela_unificada = agnts_tabela_unificada.drop_duplicates(subset=['CNPJ'])


In [ ]:
#Produtor de Obra Não Publicitária
produtor = produtores_obras_n_publi.drop_duplicates(subset=['PRODUTOR']).copy()
produtor.loc[:,'PRODUTOR_OBRAS_NAO_PUBLICITARIAS'] = True
produtor['PESSOA_FISICA'] = produtor['CNPJ_PRODUTOR'].replace({'PESSOA FÍSICA' : True}).map(lambda x: False if x != True else True)
produtor['CNPJ_NAO_INFORMADO'] = produtor['CNPJ_PRODUTOR'].replace({'NÃO INFORMADO' : True}).map(lambda x: False if x != True else True)
produtor = produtor[['PRODUTOR', 'CNPJ_PRODUTOR',
       'PRODUTOR_OBRAS_NAO_PUBLICITARIAS', 'PESSOA_FISICA',
       'CNPJ_NAO_INFORMADO']]
produtor.rename(columns={'CNPJ_PRODUTOR': 'CNPJ'}, inplace=True)


In [292]:

proj_fsa_desembolso.rename(columns={'CNPJ_PROPONENTE' : 'CNPJ'}, inplace=True)

proj_fsa_desembolso.loc[proj_fsa_desembolso['VL_TOTAL_LIBERADO'].isnull(), 'VL_TOTAL_LIBERADO'] = 'NÃO LIBERADO'
proj_fsa_desembolso.loc[proj_fsa_desembolso['DT_PRIMEIRO_DESEMBOLSO'].isnull(), 'DT_PRIMEIRO_DESEMBOLSO'] = 'NÃO LIBERADO'
proj_fsa_desembolso.loc[proj_fsa_desembolso['RAZAO_SOCIAL_DISTRIBUIDORA_PROGRAMADORA'].isnull(), 'RAZAO_SOCIAL_DISTRIBUIDORA_PROGRAMADORA'] = 'SEM DISTRIBUIDORA/PROGRAMADORA'
proj_fsa_desembolso.loc[proj_fsa_desembolso['CNPJ_DISTRIBUIDORA_PROGRAMADORA'].isnull(), 'CNPJ_DISTRIBUIDORA_PROGRAMADORA'] = 'SEM DISTRIBUIDORA/PROGRAMADORA'
proj_fsa_desembolso.loc[proj_fsa_desembolso['TITULO_PROJETO'].isnull(), 'TITULO_PROJETO'] = 'SEM TÍTULO'
proj_fsa_desembolso.loc[proj_fsa_desembolso['CNPJ_PRODUTORA'].isnull(), 'CNPJ_PRODUTORA'] = 'SEM PRODUTORA'
proj_fsa_desembolso.loc[proj_fsa_desembolso['RAZAO_SOCIAL_PRODUTORA'].isnull(), 'RAZAO_SOCIAL_PRODUTORA'] = 'SEM PRODUTORA'
